In [1]:
import os  
import json

json_data = json.load(open("cred.json"))
# print(json_data)

os.environ["MLFLOW_TRACKING_USERNAME"] = json_data["MLFLOW_TRACKING_USERNAME"]
os.environ["MLFLOW_TRACKING_PASSWORD"] = json_data["MLFLOW_TRACKING_PASSWORD"]
os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = json_data["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"]
os.environ["AWS_DEFAULT_REGION"] = json_data["AWS_DEFAULT_REGION"]
os.environ["AWS_REGION"] = json_data["AWS_REGION"]
os.environ["AWS_ACCESS_KEY_ID"] = json_data["AWS_ACCESS_KEY_ID"]
os.environ["AWS_SECRET_ACCESS_KEY"] = json_data["AWS_SECRET_ACCESS_KEY"]
os.environ["MLFLOW_S3_ENDPOINT_URL"] = json_data["MLFLOW_S3_ENDPOINT_URL"]
os.environ["MLFLOW_TRACKING_URI"] = json_data["MLFLOW_TRACKING_URI"]


from mlflow import MlflowClient

from mlflow.server import get_app_client
import mlflow.pytorch
import mlflow.keras
import mlflow
from mlflow.models import infer_signature

import matplotlib.pyplot as plt

import keras
import torch

import numpy as np
import pandas as pd

mlflow.set_experiment("my-experiment")

data_X = np.random.uniform(-1, 1, (1000, 2))
data_y = np.max(data_X, axis=1)

In [ ]:

from model_definition import ExampleModel
from lume_model.variables import ScalarInputVariable, ScalarOutputVariable

class MyModel(mlflow.pyfunc.PythonModel):
    def __init__(self, model_name, model):
        self.model_name = model_name
        self.model = model

    # this function is called when the model is loaded using pyfunc.load_model
    def predict(self, context, input, **kwargs):
        return self.model.evaluate(input)
    
    def inverse_predict_internal(self, input, **kwargs):
        return np.sqrt(input)

    def save_model(self):
        with open(f"{self.model_name}.txt", "w") as f:
            f.write("model saved")

    def load_model(self):
        with open(f"{self.model_name}.txt", "r") as f:
            return f.read()
        
    def get_lume_model(self):
        return self.model
    



input_variables = [ScalarInputVariable(name = "x1", default=0, value_range=[-100000, 1000000]), ScalarInputVariable(name = "x2", default=0, value_range=[-100000, 1000000])]
output_variables = [ScalarOutputVariable(name = "y")]
lume_model = ExampleModel(input_variables = input_variables, output_variables = output_variables)
model = MyModel("model1", lume_model)

input_sample = pd.DataFrame(data_X, columns=["x1", "x2"])

with mlflow.start_run() as run:  # you can use run_name="test1" to give a name to the run otherwise it will a random name

    
    # set some tags for the experiment
    mlflow.set_tag("exp_tag1", "exp_tag_value1")
    mlflow.set_tag("exp_tag2", "exp_tag_value2")
    mlflow.set_tag("exp_tag3", "exp_tag_value3")
    
    # model.save_model() # no need to save the model since it is saved in log_model
    mlflow.log_param("model_name", model.model_name)
    mlflow.log_param("dummy_param1", "dummy_value1")
    mlflow.log_param("dummy_param2", 0.33)
    for i in range(10):        
        mlflow.log_metric("metric1", (i / 10) ** 2 , step=i)
        mlflow.log_metric("metric2", (i / 10) ** 3 , step=i)
        mlflow.log_metric("loss", (1 / (i + 0.1) + np.random.normal(0, 0.1)) , step=i)

    # lets make some pretty graphs to store

    graph = plt.figure()
    plt.plot(range(100), [(i / 10) ** 2 for i in range(100)])
    mlflow.log_figure(graph, "figures/metric1.png")

    # alternative way to log a figure
    graph = plt.figure()
    plt.plot(range(100), [(i / 10) ** 3 for i in range(100)])
    graph.savefig("metric2.png")
    mlflow.log_artifact("metric2.png", artifact_path="figures")
    
    # i need to add depenancies:
    # lume-model @ git+https://github.com/slaclab/lume-model.git@2921e6583a6cfd49285833eb851b361aacf65b4c
    
    
    model_info = mlflow.pyfunc.log_model(
        artifact_path="model_files",
        python_model=model,
        signature=infer_signature(input_sample, model.predict(None, input = input_sample)),
        input_example=input_sample,
        code_path=["model_definition.py"],
        extra_pip_requirements=["paho-mqtt"] # example dependancy
        # registered_model_name="generic_model", # this will automatically register the model and iterate the version
     )

    # if you wanna log the model without the wrapper
    model.save_model()
    mlflow.log_artifact(
        f"{model.model_name}.txt", artifact_path="model_files_no_mlflow"
    )
    mlflow.log_artifact(
        f"pv_mapping.yaml", artifact_path="generic_model"
    )

    # set some tags

In [ ]:
# we ignore keras for now
# # keras model

# inputs = [keras.Input(name="input1", shape=(1,)), keras.Input(name="input2", shape=(1,))]
# x = keras.layers.concatenate(inputs)
# x1 = keras.layers.Dense(64, activation='relu')(x)
# x2 = keras.layers.Dense(64, activation='relu')(x1)
# outputs = keras.layers.Dense(1, name="output")(x2)
# model_keras = keras.Model(inputs=inputs, outputs=outputs)

# mlflow.keras.autolog(log_models=False)
# with mlflow.start_run() as run:
#     model_keras.compile(optimizer='adam', loss='mean_squared_error')
#     model_keras.fit([data_X[:, 0], data_X[:, 1]], data_y, epochs=10)
#     mlflow.set_tag("tag1", "tag_value1")
#     signature = infer_signature(data_X, model_keras.predict([data_X[:, 0], data_X[:, 1]]))
#     model_info_keras = mlflow.keras.log_model(model_keras, "keras_model", signature=signature)

In [ ]:
import torch
from lume_model.models import TorchModel, TorchModule


# torch model 
base_torch = torch.nn.Sequential(
    torch.nn.Linear(2,64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 1),
)

# change initialisation
for layer in base_torch:
    if isinstance(layer, torch.nn.Linear):
        torch.nn.init.xavier_uniform_(layer.weight)
        torch.nn.init.zeros_(layer.bias)
        layer.weight.data.fill_(0.05)
        layer.bias.data.fill_(0.05)
        

mlflow.pytorch.autolog()

with mlflow.start_run() as run:
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(base_torch.parameters(), lr=0.01)
    # log params
    mlflow.log_param("lr", 0.01)
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("loss", "MSELoss")
    
    for t in range(20):

        X = torch.tensor(data_X, dtype=torch.float32)
        y = torch.tensor(data_y, dtype=torch.float32)
        y_pred = base_torch(X).flatten()
        loss = criterion(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
            
        mlflow.log_metric("loss", np.mean(loss.detach().numpy()), step=t)
        print(t, np.mean(loss.detach().numpy()))
    
    mlflow.set_tag("tag1", "tag_value1")
    
    lume_torch = TorchModel(model = base_torch, input_variables = input_variables, output_variables = output_variables)

    model_torch = TorchModule(model = lume_torch)
    
    mlflow.log_artifact(
        f"torch/pv_mapping.yaml", artifact_path="torch_model"
    )
        
        
    signature = infer_signature(data_X, model_torch(torch.tensor(data_X, dtype=torch.float32)).detach().numpy()) # optional but useful

    model_info_torch = mlflow.pytorch.log_model(model_torch, "torch_model", signature=signature)

In [ ]:
# specifc model uris
print(model_info.model_uri)
# print(model_info_keras.model_uri)
print(model_info_torch.model_uri)

# lets register the models if they are not already
client = MlflowClient()
try:
    client.create_registered_model("generic_model")
    
except:
    pass

# try:
#     client.create_registered_model("keras_model")
# except:
#     pass

try:
    client.create_registered_model("torch_model")
except:
    pass

# create new model versions

# str(dict_model)
result_generic = client.create_model_version(
    name="generic_model",
    source=model_info.model_uri,
    run_id=model_info.run_id,
    tags={"tests": "pending", "framework": "pyfunc"},
)


# result_keras = client.create_model_version(
#     name="keras_model",
#     source=model_info_keras.model_uri,
#     run_id=model_info_keras.run_id,
# )


result_torch = client.create_model_version(
    name="torch_model",
    source=model_info_torch.model_uri,
    run_id=model_info_torch.run_id,
)

client.set_registered_model_alias("generic_model", "challenger", result_generic.version)


client.set_registered_model_tag("generic_model", "deployment_publish", "false")
client.set_registered_model_tag("generic_model", "deployment_type", "continuous")


# do some tests
model_ver_champ = client.get_model_version_by_alias("generic_model", "challenger").version

# 99% of the time it will pass
if np.random.uniform() < 0.99:
    client.set_model_version_tag("generic_model", f"{model_ver_champ}", "tests", "passed")
    # set the champion model
    client.set_registered_model_alias("generic_model", "champion", result_generic.version)
else:
    client.set_model_version_tag("generic_model", f"{model_ver_champ}", "tests", "failed")
    print("model failed")

# client.set_registered_model_alias("keras_model", "champion", result_keras.version)
# # client.set_registered_model_tag("keras_model", "lattice_component", "DEF")
# # client.set_registered_model_tag("keras_model", "lume_service", "false")
# # client.set_registered_model_tag("keras_model", "lume_service_url", "")
# # client.set_registered_model_tag("keras_model", "deployment_type", "continuous")
# # client.set_registered_model_tag("keras_model", "multi_model_service", "false")
# # client.set_registered_model_tag("keras_model", "retrain", "false")
# # client.set_registered_model_tag("keras_model", "retrain_endpoint", "")

client.set_registered_model_alias("torch_model", "champion", result_torch.version)
client.set_registered_model_tag("torch_model", "deployment_type", "continuous")




In [ ]:
# get model ids from registered models
genric_model  = mlflow.pyfunc.load_model(f"models:/generic_model@champion")
# keras_model = mlflow.keras.load_model(f"models:/keras_model@champion")
torch_model = mlflow.pytorch.load_model(f"models:/torch_model@champion")

# # get model info
print("Generic model")
print(genric_model, type(genric_model))
# print("Keras model")
# print(keras_model, type(keras_model)  
print("Torch model")
print(torch_model, type(torch_model))

# new_data = np.random.uniform(-1, 1, (20, 2))
# print(new_data)

# lets run some predictions

# print("Generic model")
# print(genric_model.predict(new_data))
# print("Keras model")
# print(keras_model.predict([new_data[:, 0], new_data[:, 1]]))


In [ ]:
print(genric_model.metadata.get_input_schema())

print(genric_model.metadata.get_output_schema())

# results:
# [Tensor('float64', (-1, 2))]
# [Tensor('float64', (-1,))]

In [ ]:
# # We can create a wrapper for these models into lume-models
# from lume_model.base import LUMEBaseModel
# from lume_model.variables import ScalarInputVariable, ScalarOutputVariable
# import torch
# import keras
# from test import MyModel

# input_variables = [
#     ScalarInputVariable(name="input1", default=0.1, value_range=[0.0, 1.0]),
#     ScalarInputVariable(name="input2", default=0.2, value_range=[0.0, 1.0]),
# ]
# output_variables = [
#     ScalarOutputVariable(name="output1"),
# ]

# class ExampleModel(LUMEBaseModel):
#     def evaluate(self, input_dict: dict[str, Any]) -> dict[str, Any]:
#         return {"outputs": [0]}


# m = ExampleModel(input_variables=input_variables, output_variables=output_variables)



# with open("model.pkl", "wb") as f:
#     cp.dump(m , f)

In [ ]:
# import torch

# from lume_model.models import TorchModel, TorchModule
# from lume_model.variables import ScalarInputVariable, ScalarOutputVariable

# # exemplary model definition
# base_model = torch.nn.Sequential(
#     torch.nn.Linear(2, 1),
# )

# # variable specification
# input_variables = [
#     ScalarInputVariable(name="input1", default=0.1, value_range=[0.0, 1.0]),
#     ScalarInputVariable(name="input2", default=0.2, value_range=[0.0, 1.0]),
# ]
# output_variables = [
#     ScalarOutputVariable(name="output"),
# ]

# # creation of TorchModel
# example_model = TorchModel(
#     model=base_model,
#     input_variables=input_variables,
#     output_variables=output_variables,
# )

# cp.dump(example_model, open("torch_model.pkl", "wb"))

In [ ]:
# import keras
# import numpy as np

# from lume_model.models import KerasModel
# from lume_model.variables import ScalarInputVariable, ScalarOutputVariable

# # exemplary model definition
# inputs = [keras.Input(name="input1", shape=(1,)), keras.Input(name="input2", shape=(1,))]
# outputs = keras.layers.Dense(1, activation=keras.activations.relu)(keras.layers.concatenate(inputs))
# base_model = keras.Model(inputs=inputs, outputs=outputs)

# # variable specification
# input_variables = [
#     ScalarInputVariable(name=inputs[0].name, default=0.1, value_range=[0.0, 1.0]),
#     ScalarInputVariable(name=inputs[1].name, default=0.2, value_range=[0.0, 1.0]),
# ]
# output_variables = [
#     ScalarOutputVariable(name="output"),
# ]

# # creation of KerasModel
# example_model = KerasModel(
#     model=base_model,
#     input_variables=input_variables,
#     output_variables=output_variables,
# )
# cp.dump( example_model, open("keras_model.pkl", "wb"))

In [ ]:
# from test import ExampleModel
# from lume_model.variables import ScalarInputVariable, ScalarOutputVariable
# import cloudpickle as cp

# input_variables = [
#     ScalarInputVariable(name="input1", default=0.1, value_range=[0.0, 1.0]),
#     ScalarInputVariable(name="input2", default=0.2, value_range=[0.0, 1.0]),
# ]
# output_variables = [
#     ScalarOutputVariable(name="output1"),
#     ScalarOutputVariable(name="output2"),
# ]

# m = ExampleModel(input_variables=input_variables, output_variables=output_variables)

# cp.dump(m, open("example_model.pkl", "wb"))